In [119]:
import os
import re
import numpy as np
import pandas as pd
import sklearn
import nltk

In [105]:
!conda install gensim -y

Solving environment: done

# All requested packages already installed.



In [120]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nbuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
stopword = stopwords.words('english')

In [9]:
train_data = pd.read_csv('train.csv',sep=',')

In [6]:
train_data.head(5)

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [10]:
length = len(train_data)
print(length)

38932


In [11]:
train_corpus = []
train_labels = np.zeros(length)

In [12]:
for i in range(length):
    train_corpus.append(train_data["Description"][i])
    if(train_data["Is_Response"][i]=="not happy"):
        train_labels[i] = 0
    else:
        train_labels[i] = 1

In [13]:
train_corpus[0:3]

["The room was kind of clean but had a VERY strong smell of dogs. Generally below average but ok for a overnight stay if you're not too fussy. Would consider staying again if the price was right. Breakfast was free and just about better than nothing.",
 "I stayed at the Crown Plaza April -- - April --, ----. The staff was friendly and attentive. The elevators are tiny (about -' by -'). The food in the restaurant was delicious but priced a little on the high side. Of course this is Washington DC. There is no pool and little for children to do. My room on the fifth floor had two comfortable beds and plenty of space for one person. The TV is a little small by todays standards with a limited number of channels. There was a small bit of mold in the bathtub area that could have been removed with a little bleach. It appeared the carpets were not vacummed every day. I reported a light bulb was burned out. It was never replaced. Ice machines are on the odd numbered floors, but the one on my flo

In [14]:
train_labels[0:3]

array([ 0.,  0.,  0.])

In [121]:
train = []

In [122]:
def clean_data(sentence):
    sentence = re.sub(r'[^\w\s]','',sentence)
    sentence = re.sub(r'[^a-zA-Z\s]+','',sentence)
    token_data = word_tokenize(sentence)
    new_sent = ""
    #print(token_data)
    for i in range(len(token_data)):
        if token_data[i].lower() not in stopword:
            new_sent=new_sent+token_data[i].lower()+" "
    train.append(new_sent)

In [123]:
for i in range(len(train_corpus)):
    clean_data(train_corpus[i])

In [124]:
train[0:2]

['room kind clean strong smell dogs generally average ok overnight stay youre fussy would consider staying price right breakfast free better nothing ',
 'stayed crown plaza april april staff friendly attentive elevators tiny food restaurant delicious priced little high side course washington dc pool little children room fifth floor two comfortable beds plenty space one person tv little small todays standards limited number channels small bit mold bathtub area could removed little bleach appeared carpets vacummed every day reported light bulb burned never replaced ice machines odd numbered floors one floor work encountered staff elevator one evening mentioned ice machine severel hours later maid appeared door ice two mints im sure knew room little unnerving would stay business would come vacation ']

In [125]:
vectorizer  = TfidfVectorizer(min_df=1,encoding='utf-8', max_df = 0.8, sublinear_tf=True, use_idf=True,stop_words='english')

In [126]:
train_corpus_tf_idf = vectorizer.fit_transform(train)

In [127]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [132]:
lr = LogisticRegression()
lr.fit(train_corpus_tf_idf,train_labels)
lr.score(train_corpus_tf_idf,train_labels)

0.91107572177129359

In [133]:
svmlinear = svm.SVC(kernel='linear')
svmlinear.fit(train_corpus_tf_idf,train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [134]:
linearpred = svmlinear.predict(train_corpus_tf_idf)

In [135]:
print(sum(linearpred==train_labels)/length)

0.935528613994


In [32]:
test_data = pd.read_csv('test.csv',sep=',')

In [33]:
test_data.head(3)

,User_ID,Description,Browser_Used,Device_Used
0,id80132,Looking for a motel in close proximity to TV t...,Firefox,Mobile
1,id80133,Walking distance to Madison Square Garden and ...,InternetExplorer,Desktop
2,id80134,Visited Seattle on business. Spent - nights in...,IE,Tablet


In [35]:
length_test = len(test_data)
test_labels = np.zeros(length_test)
length_test

29404

In [39]:
test_corpus = []

In [40]:
for i in range(length_test):
    test_corpus.append(test_data["Description"][i])

In [42]:
test_corpus[0:2]

["Looking for a motel in close proximity to TV taping of a Dr. Phil show, we chose the Dunes on Sunset Blvd in West Hollywood. Although the property displayed the AAA emblem, it certainly left a lot to be desired. There were chips & scrapes on the bottom of the door frame in the bathroom and the lotion containers were half full--apparently not replaced by housekeeping. We needed an early wakeup call, but couldn't use the clock radio alarm as there wasn't a radio in the room. There was no TV channel listing on the remote, or on the TV menu making viewing a chore.The TV remote had to be returned when checking-out. This place served its purpose, but not a place to revisit.",
 'Walking distance to Madison Square Garden and the Penn Subway Station. Also very close to lots of retail stores and restaurants. The hotel offers free internet, a gym and a hot breakfast. Our room was on the top floor facing --th St. The room was clean and the staff very helpful and accommodating. The downside was t

In [43]:
test_labels[0:2]

array([ 0.,  0.])

In [62]:
test = []

In [63]:
def clean_data(sentence):
    sentence = re.sub(r'[^\w\s]','',sentence)
    sentence = re.sub(r'[^a-zA-Z\s]+','',sentence)
    token_data = word_tokenize(sentence)
    new_sent = ""
    #print(token_data)
    for i in range(len(token_data)):
        if token_data[i] not in stopword:
            new_sent=new_sent+token_data[i]+" "
    test.append(new_sent)

In [64]:
for i in range(length_test):
    clean_data(test_corpus[i])

In [65]:
test[0:2]

['Looking motel close proximity TV taping Dr Phil show chose Dunes Sunset Blvd West Hollywood Although property displayed AAA emblem certainly left lot desired There chips scrapes bottom door frame bathroom lotion containers half fullapparently replaced housekeeping We needed early wakeup call couldnt use clock radio alarm wasnt radio room There TV channel listing remote TV menu making viewing choreThe TV remote returned checkingout This place served purpose place revisit ',
 'Walking distance Madison Square Garden Penn Subway Station Also close lots retail stores restaurants The hotel offers free internet gym hot breakfast Our room top floor facing th St The room clean staff helpful accommodating The downside AC making noise faucet bathroom loose things easily fixed We walked hotel Madison Square Gardens Times Square Broadway Everything considered I definitely recommend hotel friends ']

In [114]:
test_corpus_tf_idf = vectorizer.transform(test)

In [115]:
test_predict = svmlinear.predict(test_corpus_tf_idf)

In [116]:
len(test_predict)

29404

In [117]:
import csv

In [118]:
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['User_ID', 'Is_Response']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    count = 80132
    writer.writeheader()
    for predict in range(length_test):
        if(count==100000):
            num = 'id1e+05'
        else:
            num = 'id'+str(count)
        if(test_predict[predict]==1):
            label="happy"
        else:
            label="not_happy"
        writer.writerow({'User_ID': num, 'Is_Response': label})
        count+=1

In [136]:
with open('submission_linear.csv', 'w') as csvfile:
    fieldnames = ['User_ID', 'Is_Response']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    count = 80132
    writer.writeheader()
    for predict in range(length_test):
        if(count==100000):
            num = 'id1e+05'
        else:
            num = 'id'+str(count)
        if(test_predict[predict]==1):
            label="happy"
        else:
            label="not_happy"
        writer.writerow({'User_ID': num, 'Is_Response': label})
        count+=1